In [1]:
#create sparksession object
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('handling_time').getOrCreate()

In [2]:
#import Linear Regression from spark's MLlib
from pyspark.ml.regression import LinearRegression

In [3]:
#Load the dataset
df=spark.read.csv('/FileStore/tables/data_handling.csv',inferSchema=True,header=True)

In [4]:
#validate the size of data
print((df.count(), len(df.columns)))

(670857, 14)

In [5]:
#explore the data
df.printSchema()

root
-- _c0: integer (nullable = true)
-- SHP_DATE_CREATED_ID: timestamp (nullable = true)
-- SHP_DATETIME_CREATED_ID: timestamp (nullable = true)
-- SHP_DATE_HANDLING_ID: timestamp (nullable = true)
-- SHP_DATETIME_HANDLING_ID: timestamp (nullable = true)
-- SHP_STATUS_ID: string (nullable = true)
-- SHP_SENDER_ID: string (nullable = true)
-- SHP_ORDER_COST: double (nullable = true)
-- CAT_CATEG_ID_L7: string (nullable = true)
-- SHP_ADD_ZIP_CODE: integer (nullable = true)
-- SHP_DATE_SHIPPED_ID: timestamp (nullable = true)
-- SHP_DATETIME_SHIPPED_ID: timestamp (nullable = true)
-- SHP_PICKING_TYPE_ID: string (nullable = true)
-- HT_REAL: integer (nullable = true)

In [6]:
#view statistical measures of data 
df.describe().show(5)

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-3516167528173103> in <module> () 
 1 #view statistical measures of data 
 ----> 2 df . describe ( ) . show ( 5 ) 

 /databricks/spark/python/pyspark/sql/dataframe.py in describe (self, *cols) 
 1195 if len ( cols ) == 1 and isinstance ( cols [ 0 ] , list ) : 
 1196 cols = cols [ 0 ] 
 -> 1197 jdf = self . _jdf . describe ( self . _jseq ( cols ) ) 
 1198 return DataFrame ( jdf , self . sql_ctx ) 
 1199 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1255 answer = self . gateway_client . send_command ( command ) 
 1256 return_value = get_return_value(
 -> 1257 answer, self.gateway_client, self.target_id, self.name)
 1258 
 1259 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 61 def deco ( * a , ** kw ) : 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 
 65 s = e . java_exception . toString ( ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 
 330 raise Py4JError(

 Py4JJavaError : An error occurred while calling o1374.describe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 4 in stage 10.0 failed 1 times, most recent failure: Lost task 4.0 in stage 10.0 (TID 35, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
 File "/databricks/spark/python/pyspark/worker.py", line 403, in main
 process()
 File "/databricks/spark/python/pyspark/worker.py", line 398, in process
 serializer.dump_stream(func(split_index, iterator), outfile)
 File "/databricks/spark/python/pyspark/serializers.py", line 365, in dump_stream
 self.serializer.dump_stream(self._batched(iterator), stream)
 File "/databricks/spark/python/pyspark/serializers.py", line 147, in dump_stream
 for obj in iterator:
 File "/databricks/spark/python/pyspark/serializers.py", line 354, in _batched
 for item in iterator:
 File "<string>", line 1, in <lambda>
 File "/databricks/spark/python/pyspark/worker.py", line 85, in <lambda>
 return lambda *a: f(*a)
 File "/databricks/spark/python/pyspark/util.py", line 99, in wrapper
 return f(*args, **kwargs)
 File "<command-1940436074516061>", line 11, in shp_day
TypeError: strptime() argument 1 must be str, not datetime.datetime

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:490)
	at org.apache.spark.sql.execution.python.PythonUDFRunner$$anon$1.read(PythonUDFRunner.scala:81)
	at org.apache.spark.sql.execution.python.PythonUDFRunner$$anon$1.read(PythonUDFRunner.scala:64)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:444)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIterator.scala:37)
	at scala.collection.Iterator$$anon$12.hasNext(Iterator.scala:440)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage2.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$13$$anon$1.hasNext(WholeStageCodegenExec.scala:638)
	at org.apache.spark.sql.execution.aggregate.SortAggregateExec$$anonfun$doExecute$1$$anonfun$3.apply(SortAggregateExec.scala:98)
	at org.apache.spark.sql.execution.aggregate.SortAggregateExec$$anonfun$doExecute$1$$anonfun$3.apply(SortAggregateExec.scala:95)
	at org.apache.spark.rdd.RDD$$anonfun$mapPartitionsWithIndexInternal$1$$anonf

In [7]:
#sneak into the dataset
df.head(3)

Out[ 7 ]: 
[Row(_c0=533546, SHP_DATE_CREATED_ID=datetime.datetime(2017, 7, 28, 0, 0), SHP_DATETIME_CREATED_ID=datetime.datetime(2017, 7, 28, 8, 44, 1), SHP_DATE_HANDLING_ID=datetime.datetime(2017, 7, 28, 0, 0), SHP_DATETIME_HANDLING_ID=datetime.datetime(2017, 7, 28, 8, 44, 13), SHP_STATUS_ID='delivered', SHP_SENDER_ID='85ebf03e2c86c1c992a4114e4eaf6bf9', SHP_ORDER_COST=200.0, CAT_CATEG_ID_L7='412cf38b165cf0c07760390641f8aa35', SHP_ADD_ZIP_CODE=1623, SHP_DATE_SHIPPED_ID=datetime.datetime(2017, 8, 1, 0, 0), SHP_DATETIME_SHIPPED_ID=datetime.datetime(2017, 8, 1, 11, 8, 55), SHP_PICKING_TYPE_ID='drop_off', HT_REAL=50),
 Row(_c0=934098, SHP_DATE_CREATED_ID=datetime.datetime(2017, 7, 2, 0, 0), SHP_DATETIME_CREATED_ID=datetime.datetime(2017, 7, 2, 17, 9, 7), SHP_DATE_HANDLING_ID=datetime.datetime(2017, 7, 3, 0, 0), SHP_DATETIME_HANDLING_ID=datetime.datetime(2017, 7, 3, 13, 11, 59), SHP_STATUS_ID='delivered', SHP_SENDER_ID='ced3662210bef7ff33dc58f87ef49687', SHP_ORDER_COST=1151.0, CAT_CATEG_ID_L7='bf01dfb57c99031fbb0b856684643dc9', SHP_ADD_ZIP_CODE=1631, SHP_DATE_SHIPPED_ID=datetime.datetime(2017, 7, 4, 0, 0), SHP_DATETIME_SHIPPED_ID=datetime.datetime(2017, 7, 4, 19, 2, 38), SHP_PICKING_TYPE_ID='drop_off', HT_REAL=30),
 Row(_c0=998176, SHP_DATE_CREATED_ID=datetime.datetime(2017, 8, 17, 0, 0), SHP_DATETIME_CREATED_ID=datetime.datetime(2017, 8, 17, 11, 1, 34), SHP_DATE_HANDLING_ID=datetime.datetime(2017, 8, 17, 0, 0), SHP_DATETIME_HANDLING_ID=datetime.datetime(2017, 8, 17, 11, 1, 44), SHP_STATUS_ID='delivered', SHP_SENDER_ID='5fbbc02fe31c3f7668da5f38bd756911', SHP_ORDER_COST=359.88, CAT_CATEG_ID_L7='adce58cb62e031fa3e81ee43507975e0', SHP_ADD_ZIP_CODE=1425, SHP_DATE_SHIPPED_ID=datetime.datetime(2017, 8, 17, 0, 0), SHP_DATETIME_SHIPPED_ID=datetime.datetime(2017, 8, 17, 16, 39, 4), SHP_PICKING_TYPE_ID='drop_off', HT_REAL=6)]

In [8]:
# Feature Engineering
from datetime import datetime, timedelta
from pyspark.sql.functions import udf, array
from pyspark.sql.types import IntegerType

# 1. SHP_ORDER_COST_INT: Se tranforma la columna SHP_ORDER_COST de float a integer.
df = df.withColumn("SHP_ORDER_COST_INT",(df["SHP_ORDER_COST"].cast(IntegerType())))

# 2. SHP_STATUS_ID_ENC: Se realiza el encoding de la SHP_STATUS_ID para ser representada como integer.
# 3. SHP_PICKING_TYPE_ID_ENC: Se realiza el encoding de la SHP_STATUS_ID para ser representada como integer.
# 4. SHP_DAY: Se añade una columna para indicar que dia de la semana se acredito el pago.

def shp_day(x):
    return x.weekday()
  
shp_day_udf= udf(shp_day, IntegerType())

df = df.withColumn('SHP_DAY', shp_day_udf(df['SHP_DATE_HANDLING_ID']))

# 5. WKND_DAY: Se añade una columna para indicar si el pago se acredito durante el fin de semana.
def weekend_day(x):
    return 0 if x.weekday() < 5 else 1
  
weekend_day_udf= udf(weekend_day, IntegerType())

df = df.withColumn('WKND_DAY', weekend_day_udf(df['SHP_DATE_HANDLING_ID']))
df.select('WKND_DAY').show(10)

# 6. MONTH_NUM: Se añanade una columna para indicar el mes del pago.
def week_number(x):
    return x.isocalendar()[1]
  
week_number_udf= udf(week_number, IntegerType())

df = df.withColumn('WK_NUM', week_number_udf(df['SHP_DATE_HANDLING_ID']))
df.select('WK_NUM').show(10)

# 7. *WK_NUM*: Se añanade una columna para indicar la semana del año en la que se realizó el pago.
def month_number(x):
    return x.month

month_number_udf= udf(month_number, IntegerType())
  
df = df.withColumn('MONTH_NUM', month_number_udf(df['SHP_DATE_HANDLING_ID']))

# 8. *TIMESTAMP*: Se añanade un TIMESTAMP de las fechas.
def get_timestamp(x):
    return int(datetime.timestamp(x))

get_timestamp_udf = udf(get_timestamp, IntegerType())
  
df = df.withColumn('SHP_DATE_HANDLING_TIMESTAMP', get_timestamp_udf(df['SHP_DATE_HANDLING_ID']))
df = df.withColumn('SHP_DATE_CREATED_TIMESTAMP', get_timestamp_udf(df['SHP_DATE_CREATED_ID']))


In [9]:
def horas_habiles(a, b):
    """
    Retorna la diferencia en horas habiles entre dos fechas.
    No se tienen en cuenta los fines de semana.
    TODO: Anadir feriados para que no se tengan en cuenta.
    """
    start_delta = 0
    if a.weekday() < 5:
        next_day = a + timedelta(days = 1)
        next_day = next_day.replace(hour=0, minute=0, second=0)
        start_delta = (next_day - a).total_seconds()       
        start = a + timedelta(days = 1)
        start = start.replace(hour=0, minute=0, second=0)
        
    elif a.weekday() == 5:
        start = a + timedelta(days = 2)
        start = start.replace(hour=0, minute=0, second=0)

    else:
        start = a + timedelta(days = 1)
        start = start.replace(hour=0, minute=0, second=0)
    
    end = b.replace(hour=0, minute=0, second=0)
    end_delta = (b - end).total_seconds()
    
    total = start_delta + end_delta
    
    target_day = start
    while target_day <= end:
        if target_day.weekday() < 5:
            total += 86400
        target_day = target_day + timedelta(days = 1)
    total -= 24*3600
    return int(round(abs(total//3600)))

def my_handling_time(row):
    b = row[0] 
    a = row[1]
    return horas_habiles(a, b)
  
my_handling_time_udf = udf(my_handling_time, IntegerType())
  
df = df.withColumn('HT', my_handling_time_udf(array('SHP_DATETIME_SHIPPED_ID', 'SHP_DATETIME_HANDLING_ID')))
df.select('HT').show(10)

+---+
 HT|
+---+
 50|
 29|
 5|
 40|
 27|
 73|
 31|
161|
 5|
114|
+---+
only showing top 10 rows

In [10]:
from pyspark.ml.feature import StringIndexer

shp_sender_indexer =StringIndexer(inputCol="SHP_SENDER_ID", outputCol="SHP_SENDER_ID_NUM").fit(df)
df = shp_sender_indexer.transform(df)

shp_sender_indexer =StringIndexer(inputCol="CAT_CATEG_ID_L7", outputCol="CAT_CATEG_ID_L7_NUM").fit(df)
df = shp_sender_indexer.transform(df)

df.select('SHP_SENDER_ID_NUM','CAT_CATEG_ID_L7_NUM').show(3,False)

+-----------------+-------------------+
SHP_SENDER_ID_NUM|CAT_CATEG_ID_L7_NUM|
+-----------------+-------------------+
7164.0 |219.0 |
11311.0 |1501.0 |
1007.0 |3595.0 |
+-----------------+-------------------+
only showing top 3 rows

In [11]:
#import vectorassembler to create dense vectors
from pyspark.ml.linalg import Vector
from pyspark.ml.feature import VectorAssembler

In [12]:
#select the columns to create input vector
df.columns

Out[ 50 ]: 
['_c0',
 'SHP_DATE_CREATED_ID',
 'SHP_DATETIME_CREATED_ID',
 'SHP_DATE_HANDLING_ID',
 'SHP_DATETIME_HANDLING_ID',
 'SHP_STATUS_ID',
 'SHP_SENDER_ID',
 'SHP_ORDER_COST',
 'CAT_CATEG_ID_L7',
 'SHP_ADD_ZIP_CODE',
 'SHP_DATE_SHIPPED_ID',
 'SHP_DATETIME_SHIPPED_ID',
 'SHP_PICKING_TYPE_ID',
 'HT_REAL',
 'SHP_ORDER_COST_INT',
 'SHP_DAY',
 'WKND_DAY',
 'WK_NUM',
 'MONTH_NUM',
 'SHP_DATE_HANDLING_TIMESTAMP',
 'SHP_DATE_CREATED_TIMESTAMP',
 'HT',
 'SHP_SENDER_ID_NUM',
 'CAT_CATEG_ID_L7_NUM']

In [13]:
#create the vector assembler 
vec_assmebler=VectorAssembler(inputCols=['SHP_DATE_HANDLING_TIMESTAMP', 'SHP_DATE_CREATED_TIMESTAMP','SHP_SENDER_ID_NUM', 'CAT_CATEG_ID_L7_NUM', 
                    'SHP_ORDER_COST_INT', 'SHP_DAY', 'WKND_DAY', 
                    'WK_NUM', 'MONTH_NUM', 'SHP_ADD_ZIP_CODE'], outputCol='features')

In [14]:
#transform the values
features_df=vec_assmebler.transform(df)

In [15]:
#validate the presence of dense vectors 
features_df.printSchema()

root
-- _c0: integer (nullable = true)
-- SHP_DATE_CREATED_ID: timestamp (nullable = true)
-- SHP_DATETIME_CREATED_ID: timestamp (nullable = true)
-- SHP_DATE_HANDLING_ID: timestamp (nullable = true)
-- SHP_DATETIME_HANDLING_ID: timestamp (nullable = true)
-- SHP_STATUS_ID: string (nullable = true)
-- SHP_SENDER_ID: string (nullable = true)
-- SHP_ORDER_COST: double (nullable = true)
-- CAT_CATEG_ID_L7: string (nullable = true)
-- SHP_ADD_ZIP_CODE: integer (nullable = true)
-- SHP_DATE_SHIPPED_ID: timestamp (nullable = true)
-- SHP_DATETIME_SHIPPED_ID: timestamp (nullable = true)
-- SHP_PICKING_TYPE_ID: string (nullable = true)
-- HT_REAL: integer (nullable = true)
-- SHP_ORDER_COST_INT: integer (nullable = true)
-- SHP_DAY: integer (nullable = true)
-- WKND_DAY: integer (nullable = true)
-- WK_NUM: integer (nullable = true)
-- MONTH_NUM: integer (nullable = true)
-- SHP_DATE_HANDLING_TIMESTAMP: integer (nullable = true)
-- SHP_DATE_CREATED_TIMESTAMP: integer (nullable = true)
-- HT: integer (nullable = true)
-- SHP_SENDER_ID_NUM: double (nullable = false)
-- CAT_CATEG_ID_L7_NUM: double (nullable = false)
-- features: vector (nullable = true)

In [16]:
#view the details of dense vector
features_df.select('features').show(5,False)

+---------------------------------------------------------------------+
features |
+---------------------------------------------------------------------+
[1.5012E9,1.5012E9,7164.0,219.0,200.0,4.0,0.0,30.0,7.0,1623.0] |
[1.49904E9,1.4989536E9,11311.0,1501.0,1151.0,0.0,0.0,27.0,7.0,1631.0]|
[1.502928E9,1.502928E9,1007.0,3595.0,359.0,3.0,0.0,33.0,8.0,1425.0] |
[1.5027552E9,1.5027552E9,3375.0,0.0,300.0,1.0,0.0,33.0,8.0,1416.0] |
[1.502928E9,1.502928E9,18407.0,730.0,1600.0,3.0,0.0,33.0,8.0,1195.0] |
+---------------------------------------------------------------------+
only showing top 5 rows

In [17]:
from pyspark.ml.feature import MaxAbsScaler

scaler = MaxAbsScaler(inputCol="features", outputCol="scaledFeatures")

# Compute summary statistics and generate MaxAbsScalerModel
scalerModel = scaler.fit(features_df)

# rescale each feature to range [-1, 1].
scaledData = scalerModel.transform(features_df)

scaledData.select("features", "scaledFeatures").show()

+--------------------+--------------------+
 features| scaledFeatures|
+--------------------+--------------------+
[1.5012E9,1.5012E...|[0.99776042264844...|
[1.49904E9,1.4989...|[0.99632479614103...|
[1.502928E9,1.502...|[0.99890892385437...|
[1.5027552E9,1.50...|[0.99879407373377...|
[1.502928E9,1.502...|[0.99890892385437...|
[1.5005952E9,1.50...|[0.99735844722636...|
[1.5005952E9,1.50...|[0.99735844722636...|
[1.499904E9,1.499...|[0.99689904674399...|
[1.49904E9,1.4990...|[0.99632479614103...|
[1.499904E9,1.499...|[0.99689904674399...|
[1.5036192E9,1.50...|[0.99936832433674...|
[1.5008544E9,1.50...|[0.99753072240725...|
[1.5027552E9,1.50...|[0.99879407373377...|
[1.5031008E9,1.50...|[0.99902377397496...|
[1.502064E9,1.502...|[0.99833467325140...|
[1.5022368E9,1.50...|[0.99844952337199...|
[1.5037056E9,1.50...|[0.99942574939703...|
[1.5040512E9,1.50...|[0.99965544963822...|
[1.5027552E9,1.50...|[0.99879407373377...|
[1.5018912E9,1.50...|[0.99821982313081...|
+--------------------+--------------------+
only showing top 20 rows

In [18]:
#create data containing input features and output column
model_df = scaledData.select('scaledFeatures','HT')

In [19]:
model_df.show(5)

+--------------------+---+
 scaledFeatures| HT|
+--------------------+---+
[0.99776042264844...| 50|
[0.99632479614103...| 29|
[0.99890892385437...| 5|
[0.99879407373377...| 40|
[0.99890892385437...| 27|
+--------------------+---+
only showing top 5 rows

In [20]:
#size of model df
print((model_df.count(), len(model_df.columns)))

(670857, 2)

### Split Data - Train & Test sets

In [22]:
#split the data into 70/30 ratio for train test purpose
train_df,test_df=model_df.randomSplit([0.8,0.2])

In [23]:
print((train_df.count(), len(train_df.columns)))

(536241, 2)

In [24]:
print((test_df.count(), len(test_df.columns)))

(134616, 2)

In [25]:
train_df.describe().show()

+-------+-----------------+
summary| HT|
+-------+-----------------+
 count| 536241|
 mean|31.19356781745521|
 stddev|37.20479546056911|
 min| 0|
 max| 913|
+-------+-----------------+

## Build Linear Regression Model

In [27]:
#Build Linear Regression model 
lin_Reg=LinearRegression(featuresCol="scaledFeatures", labelCol='HT')

In [28]:
#fit the linear regression model on training data set 
lr_model=lin_Reg.fit(train_df)

In [29]:
lr_model.intercept

Out[ 84 ]: -206.77497404947576

In [30]:
print(lr_model.coefficients)

[9790.20703802,-9505.14316998,25.8321588788,-11.2850955237,16.7322181566,-4.29593269219,3.29062981445,-50.189214724,-7.21197545139,4.63113466896]

In [31]:
training_predictions=lr_model.evaluate(train_df)

In [32]:
training_predictions.meanSquaredError

Out[ 87 ]: 1347.3570328566047

In [33]:
training_predictions.r2

Out[ 88 ]: 0.02661273286093624

In [34]:
#make predictions on test data 
test_results=lr_model.evaluate(test_df)

In [35]:
#view the residual errors based on predictions 
test_results.residuals.show(10)

+-------------------+
 residuals|
+-------------------+
-20.727120656520412|
 78.0331027981332|
-15.383596278267532|
 -25.12959596818439|
-19.517276742465015|
-18.115343387931233|
 -19.32210991674009|
 -21.19758827327675|
 -20.16292013670042|
 25.650097286580575|
+-------------------+
only showing top 10 rows

In [36]:
#coefficient of determination value for model
test_results.r2

Out[ 91 ]: 0.024920212367780636

In [37]:
test_results.rootMeanSquaredError

Out[ 92 ]: 35.98801713927461

In [38]:
test_results.meanSquaredError

Out[ 93 ]: 1295.1373776167234